In [1]:
import dace

def vec_copy():
    sdfg = dace.SDFG("soft_hier_vec_copy")
    state = sdfg.add_state("main")
    a_host = sdfg.add_array("A", (256*32, ), dace.float16, dace.dtypes.StorageType.CPU_Heap, transient=True)
    a_dev = sdfg.add_array("soft_hier_A", (256*32, ), dace.float16, dace.dtypes.StorageType.SoftHier_HBM, transient=True)
    b_host = sdfg.add_array("B", (256*32, ), dace.float16, dace.dtypes.StorageType.CPU_Heap, transient=True)
    b_dev = sdfg.add_array("soft_hier_B", (256*32, ), dace.float16, dace.dtypes.StorageType.SoftHier_HBM, transient=True)
    ahc = state.add_access("A")
    adc = state.add_access("soft_hier_A")
    bhc = state.add_access("B")
    bdc = state.add_access("soft_hier_B")
    frag_a = sdfg.add_array("frag_A", (512, ), dace.float16, dace.dtypes.StorageType.SoftHier_TCDM, transient=True)
    frag_b = sdfg.add_array("frag_B", (512, ), dace.float16, dace.dtypes.StorageType.SoftHier_TCDM, transient=True)
    #af = state.add_access("frag_A")
    #bf = state.add_access("frag_B")

    dev_entry, dev_exit = state.add_map(name="copy_map_outer", ndrange={"i": dace.subsets.Range([(0, 256*32-1, 8192)])}, schedule=dace.dtypes.ScheduleType.SoftHier_Device)
    tblock_entry, tblock_exit = state.add_map(name="copy_map_inner", ndrange={"ii": dace.subsets.Range([(0, 256*32-1, 512)])}, schedule=dace.dtypes.ScheduleType.SoftHier_Cluster)

    #glb_to_vecin = GlobalToVECIN(name="glb_to_vecin_a", input_names=["IN_A"], output_names=["OUT_frag_A"], queue_length=1, load_length=256)
    glb_to_vecin = state.add_access("frag_A")
    #vecout_to_glb = VECOUTToGlobal(name="vecout_to_glb_b", input_names=["IN_frag_B"], output_names=["OUT_B"], queue_length=1, load_length=256)
    libnode = state.add_access("frag_B")
    #libnode = VecUnit(name="vecout_to_glb_b", input_names=["IN_frag_A"], output_names=["OUT_frag_B"], queue_length=1, load_length=256)

    state.add_edge(ahc, None, adc, None, dace.memlet.Memlet(f"A[0:{256*32}]"))
    state.add_edge(adc, None, dev_entry, "IN_A", dace.memlet.Memlet(f"soft_hier_A[0:{256*32}]"))
    state.add_edge(dev_entry, "OUT_A", tblock_entry, "IN_A", dace.memlet.Memlet(f"soft_hier_A[0:{256*32}]"))
    state.add_edge(tblock_entry, "OUT_A", glb_to_vecin, None, dace.memlet.Memlet(f"soft_hier_A[i + ii:i + ii + 512]"))
    state.add_edge(glb_to_vecin, None, libnode, None, dace.memlet.Memlet(f"frag_A[0:512]"))
    state.add_edge(libnode, None, tblock_exit, "IN_B", dace.memlet.Memlet(f"soft_hier_B[i + ii:i + ii + 512]"))
    state.add_edge(tblock_exit, "OUT_B", dev_exit, "IN_B", dace.memlet.Memlet(f"soft_hier_B[i + ii:i + ii + 512]"))
    state.add_edge(dev_exit, "OUT_B", bdc, None, dace.memlet.Memlet(f"soft_hier_B[0:{256*32}]"))
    state.add_edge(bdc, None, bhc, None, dace.memlet.Memlet(f"B[0:{256*32}]"))

    for n in [dev_entry, tblock_entry]:
        n.add_in_connector("IN_A")
        n.add_out_connector("OUT_A")

    for n in [dev_exit, tblock_exit]:
        n.add_in_connector("IN_B")
        n.add_out_connector("OUT_B")

    #libnode.add_in_connector("IN_A")
    #libnode.add_out_connector("OUT_B")

    #t = state.add_tasklet(name="assign", inputes={"_in"}, outputs={"_out"}, code="_out = _in")


    sdfg.save("soft_hier_2.sdfgz")
    return sdfg

In [2]:
s = vec_copy()

In [ ]:
from IPython.display import Code
# Code(s.generate_code()[0].clean_code, language='cpp')
# s.compile()

/scratch/dace4softhier/dace_soft_hier/dace/sdfg/validation.py:519: UserWarning: WARNING: Use of uninitialized transient "A" in state "main"
  warnings.warn('WARNING: Use of uninitialized transient "%s" in state "%s"' %
/scratch/dace4softhier/dace_soft_hier/dace/codegen/targets/soft_hier/soft_hier.py:1798: UserWarning: No `gpu_block_size` property specified on map "copy_map_outer". Falling back to the configuration entry `compiler.cuda.default_block_size`: 32,1,1. You can either specify the block size to use with the gpu_block_size property, or by adding nested `GPU_ThreadBlock` maps, which map work to individual threads. For more information, see https://spcldace.readthedocs.io/en/latest/optimization/gpu.html
  warnings.warn(


  main 0 A soft_hier_A A:None  -(A[0:8192])->  soft_hier_A:None None <dace.codegen.prettycode.CodeIOStream object at 0x7f4902d99c60>
  main 0 soft_hier_B B soft_hier_B:None  -(B[0:8192])->  B:None None <dace.codegen.prettycode.CodeIOStream object at 0x7f4902d99c60>


CompilationError: Compiler failure:
[ 50%] Built target dacestub_soft_hier_vec_copy
[35m[1mScanning dependencies of target soft_hier_vec_copy[0m
[ 75%] [32mBuilding CXX object CMakeFiles/soft_hier_vec_copy.dir/scratch/dace4softhier/dace_soft_hier/dace/soft_hier/.dacecache/soft_hier_vec_copy/src/cpu/soft_hier_vec_copy.cpp.o[0m
In file included from /scratch/dace4softhier/dace_soft_hier/dace/codegen/../runtime/include/dace/dace.h:14,
                 from /scratch/dace4softhier/dace_soft_hier/dace/soft_hier/.dacecache/soft_hier_vec_copy/src/cpu/soft_hier_vec_copy.cpp:2:
/scratch/dace4softhier/dace_soft_hier/dace/codegen/../runtime/include/dace/types.h: In constructor ‘dace::half::half(float)’:
/scratch/dace4softhier/dace_soft_hier/dace/codegen/../runtime/include/dace/types.h:93:28: warning: dereferencing type-punned pointer will break strict-aliasing rules [-Wstrict-aliasing]
             uint32_t x = *((uint32_t*)&f);
                           ~^~~~~~~~~~~~~~
/scratch/dace4softhie

In [ ]:
Code(s.generate_code()[1].clean_code, language='cpp')